In [ ]:
!pip install transformers datasets accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 85.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 83.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 18.3 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
!nvidia-smi

Wed Feb 15 05:12:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    31W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


#　dataset可至 [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/) 下載

In [ ]:
import os
path='/content/drive/MyDrive/中興大學/碩士/讀書會'
os.chdir(path)

In [ ]:
!ls

dev-v2.0.json  model_100_00  train-v2.0.json


In [ ]:
import json
from pprint import pprint
with open('train-v2.0.json','r',encoding="utf-8") as file:
  train_data = json.load(file)

for ele in train_data['data']:
  pprint(ele['paragraphs'][0])
  break

{'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born '
            'September 4, 1981) is an American singer, songwriter, record '
            'producer and actress. Born and raised in Houston, Texas, she '
            'performed in various singing and dancing competitions as a child, '
            'and rose to fame in the late 1990s as lead singer of R&B '
            "girl-group Destiny's Child. Managed by her father, Mathew "
            "Knowles, the group became one of the world's best-selling girl "
            "groups of all time. Their hiatus saw the release of Beyoncé's "
            'debut album, Dangerously in Love (2003), which established her as '
            'a solo artist worldwide, earned five Grammy Awards and featured '
            'the Billboard Hot 100 number-one singles "Crazy in Love" and '
            '"Baby Boy".',
 'qas': [{'answers': [{'answer_start': 269, 'text': 'in the late 1990s'}],
          'id': '56be85543aeaaa14008c9063',
      

In [ ]:
from pathlib import Path
def read_data(path, limit=None):
    path = Path(path)
    with open(path, 'rb') as f:
        data_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in data_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                if qa['is_impossible']== False:
                    for answer in qa['answers']:
                        contexts.append(context)
                        questions.append(question)
                        answers.append(answer['text'])
                if limit != None and len(contexts) > limit:
                  return contexts, questions, answers
                  
    return contexts, questions, answers

In [ ]:
train_contexts, train_questions, train_answers = read_data('train-v2.0.json', 10000)
eval_contexts, eval_questions, eval_answers = read_data('dev-v2.0.json',1000)

In [ ]:
print(len(train_contexts))
print(len(eval_contexts))

10001
1001


In [ ]:
print('train_contexts: ',train_contexts[0],'\n')
print('train_questions: ',train_questions[0],'\n')
print('train_answers: ',train_answers[0],'\n')

train_contexts:  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy". 

train_questions:  When did Beyonce start becoming popular? 

train_answers:  in the late 1990s 



In [ ]:
from transformers import BartTokenizer, AutoConfig
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
config = AutoConfig.from_pretrained('facebook/bart-base')

train_encodings = tokenizer(train_contexts, train_answers, text_target=train_questions, truncation=True, padding=True)#padding='max_length'
eval_encodings = tokenizer(eval_contexts, eval_answers, text_target=eval_questions, truncation=True, padding=True)

In [ ]:
train_encodings.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [ ]:
print(train_encodings['input_ids'][0])
print(tokenizer.convert_ids_to_tokens(train_encodings['input_ids'][0]))
print(tokenizer.decode(train_encodings['input_ids'][0]))

print(train_encodings['attention_mask'][0])
print(len(train_encodings['input_ids'][0]))

[0, 40401, 261, 12695, 272, 354, 6591, 10690, 1634, 12, 43732, 48229, 5605, 43621, 16948, 49066, 267, 35423, 10659, 282, 1090, 35423, 10278, 73, 19417, 12, 975, 2191, 12, 28357, 43, 36, 5400, 772, 204, 6, 14130, 43, 16, 41, 470, 3250, 6, 2214, 9408, 6, 638, 3436, 8, 3390, 4, 8912, 8, 1179, 11, 2499, 6, 1184, 6, 79, 3744, 11, 1337, 6970, 8, 7950, 9150, 25, 10, 920, 6, 8, 1458, 7, 9444, 11, 5, 628, 4525, 29, 25, 483, 3250, 9, 248, 947, 387, 1816, 12, 13839, 23313, 18, 7442, 4, 1554, 4628, 30, 69, 1150, 6, 4101, 16152, 10690, 1634, 6, 5, 333, 1059, 65, 9, 5, 232, 18, 275, 12, 11393, 1816, 1134, 9, 70, 86, 4, 2667, 25224, 794, 5, 800, 9, 12674, 12695, 18, 2453, 2642, 6, 34880, 9412, 11, 3437, 36, 35153, 238, 61, 2885, 69, 25, 10, 5540, 3025, 3612, 6, 2208, 292, 12727, 4229, 8, 3520, 5, 18919, 6003, 727, 346, 12, 1264, 7695, 22, 347, 36616, 11, 3437, 113, 8, 22, 30047, 5637, 845, 2, 2, 179, 5, 628, 4525, 29, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
import torch
class Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings

  def __getitem__(self, idx):
    return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

  def __len__(self):
    return len(self.encodings.input_ids)

In [ ]:
train_dataset = Dataset(train_encodings)
eval_dataset = Dataset(eval_encodings)

In [ ]:
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained('facebook/bart-base')

In [ ]:
from torch.utils.data import DataLoader
from tqdm.auto import tqdm, trange
import math

import transformers
from accelerate import Accelerator
from transformers import (
    AdamW,
    get_scheduler
)

In [ ]:
train_batch_size = 8      # 設定 training batch size
eval_batch_size = 8     # 設定 eval batch size
num_train_epochs = 3      # 設定 epoch

In [ ]:
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
eval_dataloader = DataLoader(eval_dataset, batch_size=eval_batch_size)

In [ ]:
learning_rate=3e-5          # 設定 learning_rate
gradient_accumulation_steps = 2   # 設定 幾步後進行反向傳播

no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },                                
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

# Scheduler and math around the number of training steps.
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)
max_train_steps = num_train_epochs * num_update_steps_per_epoch
print('max_train_steps', max_train_steps)

# scheduler
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=max_train_steps,
)

max_train_steps 1878


/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
accelerator = Accelerator()

# Prepare everything with our `accelerator`.
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [ ]:
# Train!
output_dir = 'model_100_00/'
total_batch_size = train_batch_size * accelerator.num_processes * gradient_accumulation_steps
print(f"total_batch_size = {total_batch_size}")

completed_steps = 0

for epoch in trange(num_train_epochs, desc="Epoch"):
  model.train()
  train_loss = 0
  for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
    outputs = model(**batch)
    loss = outputs.loss
    train_loss += loss.item()
    loss = loss / gradient_accumulation_steps
    accelerator.backward(loss)
    if step % gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1:
      optimizer.step()
      lr_scheduler.step()
      optimizer.zero_grad()
      completed_steps += 1
      

    if step % 50 == 0:
      print({'epoch': epoch, 'step': step, 'loss': loss.item()})
      
    
    if completed_steps >= max_train_steps:
      break

  train_loss /= len(train_dataloader)
  print(f"epoch : {epoch}, train_loss: {train_loss}")

  model.eval()
  eval_loss = 0
  for step, batch in enumerate(tqdm(eval_dataloader, desc="Eval Iteration")):
    outputs = model(**batch)
    eval_loss += outputs.loss.item()

  eval_loss /= len(eval_dataloader)
  print(f"epoch : {epoch}, eval_loss: {eval_loss}")

  if output_dir is not None:
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(f"{output_dir}epoch_{epoch}_train_loss={train_loss}_eval_loss={eval_loss}/", save_function=accelerator.save)
  

total_batch_size = 16


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1251 [00:00<?, ?it/s]

{'epoch': 0, 'step': 0, 'loss': 6.0294508934021}
{'epoch': 0, 'step': 50, 'loss': 2.4475553035736084}
{'epoch': 0, 'step': 100, 'loss': 1.4958851337432861}
{'epoch': 0, 'step': 150, 'loss': 0.8691768050193787}
{'epoch': 0, 'step': 200, 'loss': 0.5091432929039001}
{'epoch': 0, 'step': 250, 'loss': 0.6286561489105225}
{'epoch': 0, 'step': 300, 'loss': 0.4528235197067261}
{'epoch': 0, 'step': 350, 'loss': 0.266832560300827}
{'epoch': 0, 'step': 400, 'loss': 0.2600706219673157}
{'epoch': 0, 'step': 450, 'loss': 0.49230366945266724}
{'epoch': 0, 'step': 500, 'loss': 0.3553771376609802}
{'epoch': 0, 'step': 550, 'loss': 0.30182507634162903}
{'epoch': 0, 'step': 600, 'loss': 0.3634504973888397}
{'epoch': 0, 'step': 650, 'loss': 0.3939085900783539}
{'epoch': 0, 'step': 700, 'loss': 0.3051148056983948}
{'epoch': 0, 'step': 750, 'loss': 0.2342059165239334}
{'epoch': 0, 'step': 800, 'loss': 0.268934428691864}
{'epoch': 0, 'step': 850, 'loss': 0.3572200536727905}
{'epoch': 0, 'step': 900, 'loss': 

Eval Iteration:   0%|          | 0/126 [00:00<?, ?it/s]

...

epoch : 1, eval_loss: 0.8067959836787648


Iteration:   0%|          | 0/1251 [00:00<?, ?it/s]

{'epoch': 2, 'step': 0, 'loss': 0.19814664125442505}
{'epoch': 2, 'step': 50, 'loss': 0.1809288114309311}
{'epoch': 2, 'step': 100, 'loss': 0.27699822187423706}
{'epoch': 2, 'step': 150, 'loss': 0.2531451880931854}
{'epoch': 2, 'step': 200, 'loss': 0.24370037019252777}
{'epoch': 2, 'step': 250, 'loss': 0.2456999272108078}
{'epoch': 2, 'step': 300, 'loss': 0.2099996656179428}
{'epoch': 2, 'step': 350, 'loss': 0.2210041582584381}
{'epoch': 2, 'step': 400, 'loss': 0.21272946894168854}
{'epoch': 2, 'step': 450, 'loss': 0.30743545293807983}
{'epoch': 2, 'step': 500, 'loss': 0.2727924585342407}
{'epoch': 2, 'step': 550, 'loss': 0.16224370896816254}
{'epoch': 2, 'step': 600, 'loss': 0.23607729375362396}
{'epoch': 2, 'step': 650, 'loss': 0.2195415049791336}
{'epoch': 2, 'step': 700, 'loss': 0.2990144193172455}
{'epoch': 2, 'step': 750, 'loss': 0.23007570207118988}
{'epoch': 2, 'step': 800, 'loss': 0.20309609174728394}
{'epoch': 2, 'step': 850, 'loss': 0.22019967436790466}
{'epoch': 2, 'step': 

Eval Iteration:   0%|          | 0/126 [00:00<?, ?it/s]

epoch : 2, eval_loss: 0.8048683475880396


生成

In [ ]:
from transformers import AutoModelForSeq2SeqLM
from transformers import BartTokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
path = "model_100_00/epoch_2_train_loss=0.46653811364389247_eval_loss=0.8048683475880396"
model = AutoModelForSeq2SeqLM.from_pretrained(path)

# 太陽餅

In [ ]:
context = '''Bai Bingbing (May 17, 1955 -), whose real name is Bai Xuehua (formerly known as Bai Yue'e, changed her name to Bai Xuehua in January 2000), is a Taiwanese actress, singer and show host. Graduated from the Department of Life Sciences, National Aeronautical University. She is a figure in Taiwan's variety show industry and has the title of "Big Sister in the Showbiz". Started as a singer, she went to Japan to learn singing and dancing, and was called "Superstar Boxer" by Japanese magazines.'''
answer = 'Superstar Boxer'
input = tokenizer([context], [answer],truncation=True, padding=True,return_tensors="pt")

In [ ]:
question=[]
bs = 5
outputs = model.generate(
                        **input,
                        early_stopping=True,
                        num_beams=bs+3,
                        no_repeat_ngram_size=5,
                        num_return_sequences=bs,
                )
for output in outputs:
  question.append(tokenizer.decode(output, skip_special_tokens=True))
  print(tokenizer.decode(output, skip_special_tokens=True))

What was the Japanese magazine called by Bai Bingbing?
What Japanese magazine called Bai Bingbing?
What was the Japanese magazine that called Bai Bingbing?
Bai Bingbing was called what by Japanese magazines?
What was Bai Bingbing called by Japanese magazines?


# Harry Potter

In [ ]:
context = '''Harry Potter is a series of seven fantasy novels written by British author J. K. Rowling. The novels chronicle the lives of a young wizard, Harry Potter, and his friends Hermione Granger and Ron Weasley, all of whom are students at Hogwarts School of Witchcraft and Wizardry. The main story arc concerns Harry's struggle against Lord Voldemort, a dark wizard who intends to become immortal, overthrow the wizard governing body known as the Ministry of Magic and subjugate all wizards and Muggles (non-magical people).'''
#question = 'Who is the author of Harry Potter?'
answer = 'J. K. Rowling'
input = tokenizer([context], [answer],truncation=True, padding=True,return_tensors="pt")

In [ ]:
bs = 5
outputs = model.generate(
                        **input,
                        early_stopping=True,     #提早停止
                        num_beams=bs+3,          #往後延伸多少level的分支
                        no_repeat_ngram_size=5,  #5: Harry Potter is a series , 2: Harry Potter
                        num_return_sequences=bs, #回傳的句子數量
                )
for output in outputs:
  print(tokenizer.decode(output, skip_special_tokens=True))

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Who wrote Harry Potter?
Who wrote the seven fantasy novels?
Who is the author of Harry Potter?
Who wrote the Harry Potter series?
Who wrote seven fantasy novels?


# Maasai people

In [ ]:
context = '''Standing proud in the savannah with their red blankets and painted shields, the Maasai people are one
of the widely known symbols of East Africa. Their unique style, as remarked by Karen Blixen, author of
Out of Africa, “has grown from the inside, and is an expression of the race and its history.”
The Maasai are a semi-nomadic group in Kenya and northern Tanzania, wandering in bands and living
almost entirely on the meat, blood, and milk of their herds. Over the years, the fearless tribesmen have
stood strong against slavery, and resisted the urging of the Kenyan and Tanzanian governments to adopt a
more modern lifestyle. In fact, they are one of the few tribes that have retained most of their traditions. Up
until recently, the only way for a Maasai boy to achieve warrior status was to single-handedly kill a lion
with his spear.
Maasai clothing varies with age, gender, and place. The most recognizable piece of clothing is the
shúkà, a sheet of fabric worn wrapped around the body. Red is a popular color, and women generally opt
for checked, striped, or patterned pieces of cloth. Young men wear black for several months after their
circumcision, a ritual signifying their coming of age. A Maasai warrior is rarely seen without his spear and
shield. In Blixen’s words, “their weapons and finery are as much a part of their being as are a stag’s antlers”
(a male deer’s horns).
Beadwork is an important part of Maasai culture. Beaded jewelry is made by women, and is famous
for its complexity. Natural materials such as clay, shells, and ivory were used before trading with the
Europeans in the 19th century. They were then replaced by colorful glass beads, allowing for more detailed
beadwork and color patterns. Multicolored beadwork is popular among both men and women. Each color
holds a special meaning: White stands for peace, green for land and production, while red—the most
favored color among the Maasai—is the symbol of unity and bravery.'''
#question = 'Who is the author of Harry Potter?'
answer = 'Maasai people'
input = tokenizer([context], [answer],truncation=True, padding=True,return_tensors="pt")

In [ ]:
question=[]
bs = 5
outputs = model.generate(
                        **input,
                        early_stopping=True,
                        num_beams=bs+3,
                        no_repeat_ngram_size=5,
                        num_return_sequences=bs,
                )
for output in outputs:
  question.append(tokenizer.decode(output, skip_special_tokens=True))
  print(tokenizer.decode(output, skip_special_tokens=True))

What is one of the most recognizable symbols of East Africa?
Who is known for their red blankets and painted shields?
What is one of the most famous symbols of East Africa?
What is the name of the Maasai people?
What is the name of the tribe that wears red blankets?
